# Chapter 35: E-commerce Integration

Building e-commerce functionality requires balancing user experience, security, and performance. Next.js provides an ideal foundation for online stores with its ability to statically generate product catalogs for speed, dynamically handle inventory updates, and securely process payments through server-side APIs. Whether integrating with Shopify for content management or Stripe for payment processing, the App Router enables seamless commerce experiences.

By the end of this chapter, you'll master integrating Shopify's Storefront API for product management, implementing Stripe payment flows with Payment Intents, building persistent shopping carts with optimistic UI updates, handling webhooks for inventory and order synchronization, securing checkout processes against common vulnerabilities, and optimizing e-commerce performance for conversion rates.

## 35.1 Shopify Integration

Leverage Shopify's Storefront API to fetch products, collections, and manage carts while using Next.js for the presentation layer.

### Storefront API Client

```typescript
// lib/shopify/client.ts
import { createStorefrontApiClient } from '@shopify/storefront-api-client';

const client = createStorefrontApiClient({
  storeDomain: process.env.SHOPIFY_STORE_DOMAIN!,
  apiVersion: '2024-01',
  publicAccessToken: process.env.SHOPIFY_STOREFRONT_ACCESS_TOKEN!,
});

export interface Product {
  id: string;
  title: string;
  handle: string;
  description: string;
  priceRange: {
    minVariantPrice: {
      amount: string;
      currencyCode: string;
    };
  };
  images: {
    edges: Array<{
      node: {
        url: string;
        altText: string;
      };
    }>;
  };
  variants: {
    edges: Array<{
      node: {
        id: string;
        title: string;
        availableForSale: boolean;
        quantityAvailable: number;
      };
    }>;
  };
}

export async function getProducts(first = 20): Promise<Product[]> {
  const query = `
    query GetProducts($first: Int!) {
      products(first: $first) {
        edges {
          node {
            id
            title
            handle
            description
            priceRange {
              minVariantPrice {
                amount
                currencyCode
              }
            }
            images(first: 1) {
              edges {
                node {
                  url
                  altText
                }
              }
            }
          }
        }
      }
    }
  `;

  const { data } = await client.request(query, { variables: { first } });
  return data.products.edges.map((edge: any) => edge.node);
}

export async function getProduct(handle: string): Promise<Product | null> {
  const query = `
    query GetProduct($handle: String!) {
      product(handle: $handle) {
        id
        title
        handle
        description
        priceRange {
          minVariantPrice {
            amount
            currencyCode
          }
        }
        images(first: 5) {
          edges {
            node {
              url
              altText
            }
          }
        }
        variants(first: 10) {
          edges {
            node {
              id
              title
              availableForSale
              quantityAvailable
              price {
                amount
                currencyCode
              }
            }
          }
        }
      }
    }
  `;

  const { data } = await client.request(query, { variables: { handle } });
  return data.product;
}

// Cart management
export async function createCart(): Promise<string> {
  const mutation = `
    mutation CreateCart {
      cartCreate {
        cart {
          id
          checkoutUrl
        }
      }
    }
  `;
  
  const { data } = await client.request(mutation);
  return data.cartCreate.cart.id;
}

export async function addToCart(cartId: string, variantId: string, quantity: number) {
  const mutation = `
    mutation AddToCart($cartId: ID!, $lines: [CartLineInput!]!) {
      cartLinesAdd(cartId: $cartId, lines: $lines) {
        cart {
          id
          lines(first: 10) {
            edges {
              node {
                id
                quantity
                merchandise {
                  ... on ProductVariant {
                    id
                    title
                    product {
                      title
                    }
                  }
                }
              }
            }
          }
          cost {
            totalAmount {
              amount
              currencyCode
            }
          }
        }
      }
    }
  `;

  return await client.request(mutation, {
    variables: {
      cartId,
      lines: [{ merchandiseId: variantId, quantity }],
    },
  });
}
```

### Product Display Components

```typescript
// app/shop/page.tsx
import { getProducts } from '@/lib/shopify/client';
import Image from 'next/image';
import Link from 'next/link';
import { unstable_cache } from 'next/cache';

const getCachedProducts = unstable_cache(
  async () => getProducts(24),
  ['shopify-products'],
  { revalidate: 3600 } // Revalidate every hour
);

export const metadata = {
  title: 'Shop | Next.js Mastery',
};

export default async function ShopPage() {
  const products = await getCachedProducts();

  return (
    <div className="max-w-7xl mx-auto px-4 py-12">
      <h1 className="text-4xl font-bold mb-8">All Products</h1>
      
      <div className="grid grid-cols-1 sm:grid-cols-2 lg:grid-cols-3 xl:grid-cols-4 gap-8">
        {products.map((product) => (
          <ProductCard key={product.id} product={product} />
        ))}
      </div>
    </div>
  );
}

function ProductCard({ product }: { product: any }) {
  const image = product.images.edges[0]?.node;
  const price = product.priceRange.minVariantPrice;

  return (
    <Link href={`/shop/${product.handle}`} className="group">
      <div className="aspect-square relative bg-gray-100 rounded-lg overflow-hidden mb-4">
        {image && (
          <Image
            src={image.url}
            alt={image.altText || product.title}
            fill
            className="object-cover group-hover:scale-105 transition-transform"
            sizes="(max-width: 768px) 100vw, (max-width: 1200px) 50vw, 25vw"
          />
        )}
      </div>
      <h2 className="font-semibold text-lg mb-1">{product.title}</h2>
      <p className="text-gray-600">
        {new Intl.NumberFormat('en-US', {
          style: 'currency',
          currency: price.currencyCode,
        }).format(parseFloat(price.amount))}
      </p>
    </Link>
  );
}
```

```typescript
// app/shop/[handle]/page.tsx
import { getProduct } from '@/lib/shopify/client';
import { notFound } from 'next/navigation';
import Image from 'next/image';
import { AddToCartButton } from '@/components/shop/add-to-cart';
import { Metadata } from 'next';

interface Props {
  params: { handle: string };
}

export async function generateMetadata({ params }: Props): Promise<Metadata> {
  const product = await getProduct(params.handle);
  if (!product) return {};
  
  return {
    title: `${product.title} | Shop`,
    description: product.description.slice(0, 160),
  };
}

export default async function ProductPage({ params }: Props) {
  const product = await getProduct(params.handle);
  
  if (!product) {
    notFound();
  }

  const images = product.images.edges;
  const variants = product.variants.edges;

  return (
    <div className="max-w-7xl mx-auto px-4 py-12">
      <div className="grid grid-cols-1 md:grid-cols-2 gap-12">
        {/* Image Gallery */}
        <div className="space-y-4">
          {images.map((img: any, idx: number) => (
            <div key={idx} className="relative aspect-square rounded-lg overflow-hidden">
              <Image
                src={img.node.url}
                alt={img.node.altText || product.title}
                fill
                className="object-cover"
                priority={idx === 0}
                sizes="(max-width: 768px) 100vw, 50vw"
              />
            </div>
          ))}
        </div>

        {/* Product Info */}
        <div className="space-y-6">
          <h1 className="text-3xl font-bold">{product.title}</h1>
          
          <p className="text-2xl text-gray-900">
            {new Intl.NumberFormat('en-US', {
              style: 'currency',
              currency: product.priceRange.minVariantPrice.currencyCode,
            }).format(parseFloat(product.priceRange.minVariantPrice.amount))}
          </p>

          <div className="prose text-gray-600">
            {product.description}
          </div>

          <AddToCartButton 
            variants={variants.map((v: any) => v.node)} 
            productTitle={product.title}
          />
        </div>
      </div>
    </div>
  );
}
```

## 35.2 Stripe Integration

Implement secure payment processing with Stripe's Payment Intents API for custom checkout experiences.

### Payment Intent API

```typescript
// app/api/payment-intent/route.ts
import { NextRequest, NextResponse } from 'next/server';
import Stripe from 'stripe';

const stripe = new Stripe(process.env.STRIPE_SECRET_KEY!, {
  apiVersion: '2024-06-20',
});

export async function POST(req: NextRequest) {
  try {
    const { amount, items, customer_email } = await req.json();

    // Calculate amount on server to prevent tampering
    const calculatedAmount = await calculateOrderAmount(items);

    if (amount !== calculatedAmount) {
      return NextResponse.json(
        { error: 'Amount mismatch' },
        { status: 400 }
      );
    }

    const paymentIntent = await stripe.paymentIntents.create({
      amount: calculatedAmount,
      currency: 'usd',
      automatic_payment_methods: { enabled: true },
      metadata: {
        items: JSON.stringify(items.map((i: any) => ({ id: i.id, qty: i.quantity }))),
      },
      receipt_email: customer_email,
    });

    return NextResponse.json({
      clientSecret: paymentIntent.client_secret,
    });
  } catch (error) {
    console.error('Payment intent error:', error);
    return NextResponse.json(
      { error: 'Failed to create payment intent' },
      { status: 500 }
    );
  }
}

async function calculateOrderAmount(items: any[]): Promise<number> {
  // Fetch prices from database or CMS to prevent client-side manipulation
  const lineItems = await Promise.all(
    items.map(async (item) => {
      const product = await getProductFromDB(item.id);
      return product.price * item.quantity * 100; // Convert to cents
    })
  );
  
  return lineItems.reduce((a, b) => a + b, 0);
}
```

### Checkout Form with Stripe Elements

```typescript
// components/checkout/checkout-form.tsx
'use client';

import { useState, useEffect } from 'react';
import {
  PaymentElement,
  useStripe,
  useElements,
} from '@stripe/react-stripe-js';
import { StripeError } from '@stripe/stripe-js';

interface CheckoutFormProps {
  clientSecret: string;
  returnUrl: string;
}

export function CheckoutForm({ clientSecret, returnUrl }: CheckoutFormProps) {
  const stripe = useStripe();
  const elements = useElements();
  const [isLoading, setIsLoading] = useState(false);
  const [errorMessage, setErrorMessage] = useState<string | null>(null);

  const handleSubmit = async (e: React.FormEvent) => {
    e.preventDefault();

    if (!stripe || !elements) {
      return;
    }

    setIsLoading(true);
    setErrorMessage(null);

    const { error, paymentIntent } = await stripe.confirmPayment({
      elements,
      confirmParams: {
        return_url: returnUrl,
      },
      redirect: 'if_required',
    });

    if (error) {
      setErrorMessage(error.message || 'An unexpected error occurred.');
    } else if (paymentIntent && paymentIntent.status === 'succeeded') {
      // Handle successful payment (show success message, clear cart)
      window.location.href = `/order/success?payment_intent=${paymentIntent.id}`;
    }

    setIsLoading(false);
  };

  return (
    <form onSubmit={handleSubmit} className="space-y-6">
      <div className="p-4 border rounded-lg bg-white">
        <PaymentElement 
          options={{
            layout: 'tabs',
            defaultValues: {
              billingDetails: {
                email: '', // Pre-fill if user is logged in
              },
            },
          }}
        />
      </div>

      {errorMessage && (
        <div className="p-4 bg-red-50 text-red-600 rounded-lg" role="alert">
          {errorMessage}
        </div>
      )}

      <button
        disabled={isLoading || !stripe || !elements}
        className="w-full py-3 px-4 bg-blue-600 text-white font-semibold rounded-lg hover:bg-blue-700 disabled:opacity-50 disabled:cursor-not-allowed"
      >
        {isLoading ? (
          <span className="flex items-center justify-center gap-2">
            <Spinner className="w-5 h-5 animate-spin" />
            Processing...
          </span>
        ) : (
          'Pay Now'
        )}
      </button>
    </form>
  );
}
```

```typescript
// app/checkout/page.tsx
'use client';

import { useEffect, useState } from 'react';
import { loadStripe } from '@stripe/stripe-js';
import { Elements } from '@stripe/react-stripe-js';
import { CheckoutForm } from '@/components/checkout/checkout-form';
import { useCart } from '@/hooks/use-cart';

const stripePromise = loadStripe(process.env.NEXT_PUBLIC_STRIPE_PUBLISHABLE_KEY!);

export default function CheckoutPage() {
  const [clientSecret, setClientSecret] = useState('');
  const { items, total } = useCart();

  useEffect(() => {
    // Create payment intent as soon as page loads
    fetch('/api/payment-intent', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({
        items,
        amount: total * 100, // cents
      }),
    })
      .then((res) => res.json())
      .then((data) => setClientSecret(data.clientSecret));
  }, [items, total]);

  if (!clientSecret) {
    return <div className="p-12 text-center">Loading checkout...</div>;
  }

  return (
    <div className="max-w-2xl mx-auto px-4 py-12">
      <h1 className="text-3xl font-bold mb-8">Checkout</h1>
      
      <Elements stripe={stripePromise} options={{ clientSecret }}>
        <CheckoutForm 
          clientSecret={clientSecret}
          returnUrl={`${window.location.origin}/order/confirmation`}
        />
      </Elements>
    </div>
  );
}
```

## 35.3 Shopping Cart Implementation

Build a persistent cart with optimistic UI updates and server synchronization.

### Cart Context with Persistence

```typescript
// hooks/use-cart.tsx
'use client';

import { createContext, useContext, useState, useEffect, ReactNode } from 'react';
import { useOptimistic } from 'react';

interface CartItem {
  id: string;
  variantId: string;
  title: string;
  price: number;
  quantity: number;
  image: string;
}

interface CartContextType {
  items: CartItem[];
  addItem: (item: Omit<CartItem, 'quantity'>, quantity?: number) => void;
  removeItem: (variantId: string) => void;
  updateQuantity: (variantId: string, quantity: number) => void;
  clearCart: () => void;
  total: number;
  itemCount: number;
  isOpen: boolean;
  setIsOpen: (open: boolean) => void;
}

const CartContext = createContext<CartContextType | undefined>(undefined);

export function CartProvider({ children }: { children: ReactNode }) {
  const [items, setItems] = useState<CartItem[]>([]);
  const [isOpen, setIsOpen] = useState(false);
  const [optimisticItems, addOptimisticItem] = useOptimistic(
    items,
    (state, newItem: CartItem) => [...state, newItem]
  );

  // Load from localStorage on mount
  useEffect(() => {
    const saved = localStorage.getItem('cart');
    if (saved) {
      try {
        setItems(JSON.parse(saved));
      } catch (e) {
        console.error('Failed to parse cart', e);
      }
    }
  }, []);

  // Persist to localStorage
  useEffect(() => {
    localStorage.setItem('cart', JSON.stringify(items));
  }, [items]);

  const addItem = async (newItem: Omit<CartItem, 'quantity'>, quantity = 1) => {
    // Optimistic update
    addOptimisticItem({ ...newItem, quantity });
    
    setItems((current) => {
      const existing = current.find((i) => i.variantId === newItem.variantId);
      if (existing) {
        return current.map((i) =>
          i.variantId === newItem.variantId
            ? { ...i, quantity: i.quantity + quantity }
            : i
        );
      }
      return [...current, { ...newItem, quantity }];
    });

    // Sync with Shopify cart (optional)
    await syncWithShopifyCart(newItem.variantId, quantity);
  };

  const removeItem = (variantId: string) => {
    setItems((current) => current.filter((i) => i.variantId !== variantId));
  };

  const updateQuantity = (variantId: string, quantity: number) => {
    if (quantity < 1) {
      removeItem(variantId);
      return;
    }
    setItems((current) =>
      current.map((i) =>
        i.variantId === variantId ? { ...i, quantity } : i
      )
    );
  };

  const clearCart = () => setItems([]);

  const total = items.reduce((sum, item) => sum + item.price * item.quantity, 0);
  const itemCount = items.reduce((sum, item) => sum + item.quantity, 0);

  return (
    <CartContext.Provider
      value={{
        items,
        addItem,
        removeItem,
        updateQuantity,
        clearCart,
        total,
        itemCount,
        isOpen,
        setIsOpen,
      }}
    >
      {children}
    </CartContext.Provider>
  );
}

async function syncWithShopifyCart(variantId: string, quantity: number) {
  // Call your API to sync with Shopify's cart
  await fetch('/api/cart/add', {
    method: 'POST',
    headers: { 'Content-Type': 'application/json' },
    body: JSON.stringify({ variantId, quantity }),
  });
}

export const useCart = () => {
  const context = useContext(CartContext);
  if (!context) throw new Error('useCart must be used within CartProvider');
  return context;
};
```

```typescript
// components/cart/cart-drawer.tsx
'use client';

import { useCart } from '@/hooks/use-cart';
import Image from 'next/image';
import Link from 'next/link';

export function CartDrawer() {
  const { items, isOpen, setIsOpen, removeItem, updateQuantity, total } = useCart();

  return (
    <>
      {/* Backdrop */}
      {isOpen && (
        <div 
          className="fixed inset-0 bg-black/50 z-40"
          onClick={() => setIsOpen(false)}
        />
      )}
      
      {/* Drawer */}
      <div className={`fixed top-0 right-0 h-full w-full max-w-md bg-white shadow-xl z-50 transform transition-transform ${
        isOpen ? 'translate-x-0' : 'translate-x-full'
      }`}>
        <div className="flex flex-col h-full">
          <div className="flex items-center justify-between p-4 border-b">
            <h2 className="text-lg font-semibold">Shopping Cart ({items.length})</h2>
            <button 
              onClick={() => setIsOpen(false)}
              className="p-2 hover:bg-gray-100 rounded-full"
              aria-label="Close cart"
            >
              <CloseIcon className="w-5 h-5" />
            </button>
          </div>

          <div className="flex-1 overflow-y-auto p-4 space-y-4">
            {items.length === 0 ? (
              <div className="text-center py-12 text-gray-500">
                Your cart is empty
              </div>
            ) : (
              items.map((item) => (
                <div key={item.variantId} className="flex gap-4">
                  <div className="relative w-20 h-20 rounded-lg overflow-hidden bg-gray-100 flex-shrink-0">
                    <Image
                      src={item.image}
                      alt={item.title}
                      fill
                      className="object-cover"
                    />
                  </div>
                  
                  <div className="flex-1">
                    <h3 className="font-medium text-sm">{item.title}</h3>
                    <p className="text-gray-600 text-sm">
                      ${(item.price / 100).toFixed(2)}
                    </p>
                    
                    <div className="flex items-center gap-2 mt-2">
                      <button
                        onClick={() => updateQuantity(item.variantId, item.quantity - 1)}
                        className="w-8 h-8 rounded border flex items-center justify-center hover:bg-gray-50"
                        aria-label="Decrease quantity"
                      >
                        -
                      </button>
                      <span className="w-8 text-center">{item.quantity}</span>
                      <button
                        onClick={() => updateQuantity(item.variantId, item.quantity + 1)}
                        className="w-8 h-8 rounded border flex items-center justify-center hover:bg-gray-50"
                        aria-label="Increase quantity"
                      >
                        +
                      </button>
                    </div>
                  </div>
                  
                  <button
                    onClick={() => removeItem(item.variantId)}
                    className="text-red-500 hover:text-red-700 text-sm"
                    aria-label="Remove item"
                  >
                    Remove
                  </button>
                </div>
              ))
            )}
          </div>

          {items.length > 0 && (
            <div className="border-t p-4 space-y-4">
              <div className="flex justify-between text-lg font-semibold">
                <span>Total</span>
                <span>${(total / 100).toFixed(2)}</span>
              </div>
              
              <Link
                href="/checkout"
                onClick={() => setIsOpen(false)}
                className="block w-full py-3 bg-blue-600 text-white text-center rounded-lg hover:bg-blue-700 font-medium"
              >
                Proceed to Checkout
              </Link>
            </div>
          )}
        </div>
      </div>
    </>
  );
}
```

## 35.4 Webhooks and Order Management

Handle post-purchase events like payment confirmation, inventory updates, and fulfillment.

### Stripe Webhook Handler

```typescript
// app/api/webhooks/stripe/route.ts
import { NextRequest, NextResponse } from 'next/server';
import Stripe from 'stripe';
import { db } from '@/lib/db';
import { resend } from '@/lib/email/resend';

const stripe = new Stripe(process.env.STRIPE_SECRET_KEY!, {
  apiVersion: '2024-06-20',
});
const webhookSecret = process.env.STRIPE_WEBHOOK_SECRET!;

export async function POST(req: NextRequest) {
  const payload = await req.text();
  const signature = req.headers.get('stripe-signature');

  if (!signature) {
    return NextResponse.json({ error: 'Missing signature' }, { status: 400 });
  }

  let event: Stripe.Event;

  try {
    event = stripe.webhooks.constructEvent(payload, signature, webhookSecret);
  } catch (err: any) {
    console.error('Webhook signature verification failed:', err.message);
    return NextResponse.json({ error: 'Invalid signature' }, { status: 400 });
  }

  try {
    switch (event.type) {
      case 'payment_intent.succeeded': {
        const paymentIntent = event.data.object as Stripe.PaymentIntent;
        await handlePaymentSuccess(paymentIntent);
        break;
      }
      
      case 'payment_intent.payment_failed': {
        const paymentIntent = event.data.object as Stripe.PaymentIntent;
        await handlePaymentFailure(paymentIntent);
        break;
      }
      
      case 'charge.refunded': {
        const charge = event.data.object as Stripe.Charge;
        await handleRefund(charge);
        break;
      }
      
      default:
        console.log(`Unhandled event type: ${event.type}`);
    }

    return NextResponse.json({ received: true });
  } catch (error) {
    console.error('Webhook processing error:', error);
    return NextResponse.json(
      { error: 'Webhook processing failed' },
      { status: 500 }
    );
  }
}

async function handlePaymentSuccess(paymentIntent: Stripe.PaymentIntent) {
  // Idempotency: Check if order already exists
  const existingOrder = await db.order.findUnique({
    where: { paymentIntentId: paymentIntent.id },
  });
  
  if (existingOrder) {
    console.log('Order already processed:', existingOrder.id);
    return;
  }

  // Parse metadata
  const items = JSON.parse(paymentIntent.metadata.items || '[]');
  
  // Create order in database
  const order = await db.order.create({
    data: {
      paymentIntentId: paymentIntent.id,
      amount: paymentIntent.amount,
      currency: paymentIntent.currency,
      status: 'paid',
      customerEmail: paymentIntent.receipt_email,
      items: {
        create: items.map((item: any) => ({
          productId: item.id,
          quantity: item.qty,
          // Fetch current price to prevent manipulation
        })),
      },
    },
  });

  // Send confirmation email
  if (order.customerEmail) {
    await resend.emails.send({
      from: 'orders@example.com',
      to: order.customerEmail,
      subject: `Order Confirmation #${order.id}`,
      react: OrderConfirmationEmail({ order }),
    });
  }

  // Update inventory
  await updateInventory(items);

  // Clear cart if session ID exists
  if (paymentIntent.metadata.cartId) {
    await clearShopifyCart(paymentIntent.metadata.cartId);
  }
}

async function handlePaymentFailure(paymentIntent: Stripe.PaymentIntent) {
  await db.failedPayment.create({
    data: {
      paymentIntentId: paymentIntent.id,
      amount: paymentIntent.amount,
      errorMessage: paymentIntent.last_payment_error?.message,
      customerEmail: paymentIntent.receipt_email,
    },
  });
  
  // Send abandoned cart email if email exists
  if (paymentIntent.receipt_email) {
    // Schedule recovery email
  }
}
```

## Key Takeaways from Chapter 35

1. **Shopify Integration**: Use the Storefront API with public access tokens for read operations (products, collections) and the Admin API with private tokens for mutations (inventory updates). Implement cart management using Shopify's Cart API or maintain local state synchronized via webhooks.

2. **Stripe Security**: Always calculate order totals server-side in Payment Intent creation to prevent price manipulation. Use webhook signature verification (`stripe.webhooks.constructEvent`) to ensure event authenticity. Implement idempotency checks to prevent duplicate order processing.

3. **Cart Persistence**: Use React Context for cart state management with localStorage hydration for guest users. Implement optimistic updates for immediate UI feedback, then synchronize with backend cart APIs. Store cart IDs in cookies or localStorage to maintain cart across sessions.

4. **Checkout Flow**: Implement Stripe Elements for PCI-compliant payment collection without handling raw card data. Use Payment Intents for dynamic payment methods (cards, wallets, buy-now-pay-later) and confirmation flows. Provide clear error handling for failed payments with retry mechanisms.

5. **Webhook Reliability**: Configure Stripe webhooks to handle idempotency using database unique constraints on `paymentIntentId`. Implement retry logic for failed webhook processing and use queue systems (Redis, SQS) for high-volume order processing to prevent data loss.

6. **Performance Optimization**: Statically generate product pages with ISR for fast loading, using `unstable_cache` for Shopify API responses. Implement image optimization with `next/image` and Shopify's CDN URL parameters. Use Edge Functions for cart operations to minimize latency.

7. **Order Management**: Create comprehensive order records linking payment intents to line items for reconciliation. Send confirmation emails asynchronously via services like Resend or SendGrid. Update inventory atomically with order creation to prevent overselling in high-traffic scenarios.

## Coming Up Next

**Chapter 36: File Handling & Uploads**

With e-commerce functionality in place, your application likely needs to handle user-generated content and file uploads. In Chapter 36, we'll explore secure file upload strategies, direct-to-S3 upload patterns, image processing and optimization, virus scanning, upload progress tracking, and handling large file transfers. You'll learn how to build robust file management systems that scale without overwhelming your application servers.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='34. content_management.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='36. file_handling_and_uploads.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
